In [2]:
from __future__ import division, print_function

import numpy as np
from scipy.sparse import random
from scipy import stats
from numpy.random import default_rng
from pylab import plt

In [3]:
def J(Y, lamb):
    return max(np.linalg.norm(Y, ord=2), np.linalg.norm(Y, np.inf)/lamb)

In [184]:
class R_pca:

    def __init__(self, D, Y, mu=None, lmbda=None, rho=None):
        self.D = D
        self.E = np.zeros(self.D.shape)
        # self.Y = np.zeros(self.D.shape)
        self.Y = Y
        self.rho = rho

        if mu:
            self.mu = mu
        else:
            self.mu = np.prod(self.D.shape) / (4 * self.norm_p(self.D, 2))  

        self.mu_inv = 1 / self.mu

        if lmbda:
            self.lmbda = lmbda
        else:
            self.lmbda = 1 / np.sqrt(32*np.max(self.D.shape))

    @staticmethod
    def norm_p(M, p):
        return np.sum(np.power(M, p))

    @staticmethod
    def shrink(M, tau):
        return np.sign(M) * np.maximum((np.abs(M) - tau), np.zeros(M.shape))

    def svd_threshold(self, M, tau):
        U, S, V = np.linalg.svd(M, full_matrices=False)
        return np.dot(U, np.dot(np.diag(self.shrink(S, tau)), V))

    def fit(self, tol=None, max_iter=1000, iter_print=100):
        iter = 0
        err = np.Inf
        Ek = self.E
        Yk = self.Y
        Ak = np.zeros(self.D.shape)

        if tol:
            _tol = tol
        else:
            _tol = 1E-7 * self.norm_p(np.abs(self.D), 2)

        while (err > _tol) and iter < max_iter:
            Ak = self.svd_threshold(self.D - Ek + self.mu_inv * Yk, self.mu_inv)
            Ek = self.shrink(self.D - Ak + (self.mu_inv * Yk), self.mu_inv * self.lmbda)
            Yk = Yk + self.mu * (self.D - Ak - Ek)
            err = self.norm_p(np.abs(self.D - Ak - Ek), 2)
            self.mu *= self.rho
            iter += 1

        np.fill_diagonal(Ak, 1)
        np.fill_diagonal(Ek, 0)

        self.A = Ak
        self.E = Ek
        return Ak, Ek

In [255]:
def InexactALM(D, lamb, mu, rho):
    Y0 = D/J(D, lamb)
    # Y0 = np.zeros(D.shape)
    rpca = R_pca(D, Y0, mu, lamb, rho)
    A, E = rpca.fit()
    return (A, E)

In [268]:
def Clustering(D):
    n = len(D)
    A = np.zeros(D.shape)
    lamb = 1/(32*np.sqrt(n))
    i = 0
    while(i < 10):
        A, E = InexactALM(D+np.identity(n), lamb, mu=1.4, rho=1.1)
        if(A.trace() > n):
            lamb /= 2
            # print("0")
        elif(A.trace() < n):
            lamb *= 1
            # print("1")
        else:
            pass
        i += 1
    A = np.around(A)
    E = np.around(E)
    A = A.astype(int)
    E = E.astype(int)
    print(A)
    print(E)
    print("Rank of A:", np.linalg.matrix_rank(A))
    print("Rank of E: ", np.linalg.matrix_rank(E))

In [275]:
if __name__=="__main__":
    n = 6
    p = 0.8
    A1 = np.random.binomial(n=1, p=p, size=(n,n))
    A = (A1 + A1.T)/2
    A = np.around(A)
    A = A.astype(int)
    np.fill_diagonal(A, 0)
    print(A)
    print(np.linalg.matrix_rank(A))
    Clustering(A)

[[0 1 0 1 1 0]
 [1 0 1 0 0 0]
 [0 1 0 1 0 1]
 [1 0 1 0 0 1]
 [1 0 0 0 0 1]
 [0 0 1 1 1 0]]
6
[[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 1 0 1]
 [0 0 1 1 0 1]
 [0 0 0 0 1 0]
 [0 0 1 1 0 1]]
[[0 1 0 1 1 0]
 [1 0 1 0 0 0]
 [0 1 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 1]
 [0 0 0 0 1 0]]
Rank of A: 4
Rank of E:  6


In [276]:
def is_block_diagonal(matrix):
    n = len(matrix)
    block_size = None
    col_starts = []
    for i in range(n):
        if block_size is None:
            block_size = len(matrix[i])
        elif len(matrix[i]) != block_size:
            return False
        col_starts.append(i*block_size)
    for i in range(n):
        start = col_starts[i]
        end = start + block_size
        for j in range(start, end):
            if matrix[i][j-start] != 0 and any(matrix[k][j-start] != 0 for k in range(n) if k != i):
                return False
    return True


In [281]:
p = 0.8
A1 = np.random.binomial(n=1, p=p, size=(n,n))
A = (A1 + A1.T)/2
A = np.around(A)
A = A.astype(int)
np.fill_diagonal(A, 0)
A = [[1, 1, 0, 0], [1, 1, 0, 0], [0, 0, 1, 1], [0, 0, 1, 1]]
print(A)
print(is_block_diagonal(A))

[[1, 1, 0, 0], [1, 1, 0, 0], [0, 0, 1, 1], [0, 0, 1, 1]]
False
